In [44]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [45]:
# Make use of the colorline recipe (found at https://nbviewer.jupyter.org/github/dpsanders/matplotlib-examples/blob/master/colorline.ipynb)

def make_segments(x, y):
    '''
    Create list of line segments from x and y coordinates, in the correct format for LineCollection:
    an array of the form   numlines x (points per line) x 2 (x and y) array
    '''

    points = np.array([x, y]).T.reshape(-1, 1, 2)
    segments = np.concatenate([points[:-1], points[1:]], axis=1)
    
    return segments

def colorline(x, y, z=None, cmap=plt.get_cmap('copper'), norm=plt.Normalize(0.0, 1.0), linewidth=3, alpha=1.0):
    '''
    Plot a colored line with coordinates x and y
    Optionally specify colors in the array z
    Optionally specify a colormap, a norm function and a line width
    '''
    
    # Default colors equally spaced on [0,1]:
    if z is None:
        z = np.linspace(0.0, 1.0, len(x))
           
    # Special case if a single number:
    if not hasattr(z, "__iter__"):  # to check for numerical input -- this is a hack
        z = np.array([z])
        
    z = np.asarray(z)
    
    segments = make_segments(x, y)
    lc = LineCollection(segments, array=z, cmap=cmap, norm=norm, linewidth=linewidth, alpha=alpha)
    
    ax = plt.gca()
    ax.add_collection(lc)
    
    return lc

In [46]:
def reorientAxes(mass_1_x, mass_1_y, mass_2_x, mass_2_y, moving_mass_x, moving_mass_y):
    print(
        f"original: m1: ({mass_1_x}, {mass_1_y}), m2: ({mass_2_x}, {mass_2_y}), m: ({moving_mass_x}, {moving_mass_y})")

    # Reposition the masses such that M1 is at the origin and M2 lies along the y-axis
    # First, move all points
    # Then rotate

    mass_2_x -= mass_1_x
    mass_2_y -= mass_1_y
    moving_mass_x -= mass_1_x
    moving_mass_y -= mass_1_y

    m_theta = np.arctan2(moving_mass_y, moving_mass_x)
    m2_theta = np.arctan2(mass_2_y, mass_2_x)

    mass_2_x = np.sqrt(mass_2_x ** 2 + mass_2_y ** 2)
    mass_2_y = 0
    mass_1_x = 0
    mass_1_y = 0

    # Subtract m2_theta from current m_theta

    m_theta -= m2_theta
    m_r = np.sqrt(moving_mass_x ** 2 + moving_mass_y ** 2)

    # Convert back to cartesian coordinates

    moving_mass_x = m_r * np.cos(m_theta)
    moving_mass_y = m_r * np.sin(m_theta)

    # m1 and m2 should now lay along the x-axis and m should be rotated relative to them
    print(
        f"final: m1: ({mass_1_x}, {mass_1_y}), m2: ({mass_2_x}, {mass_2_y}), m: ({moving_mass_x}, {moving_mass_y})")

    return (mass_1_x, mass_1_y, mass_2_x, mass_2_y, moving_mass_x, moving_mass_y)

In [47]:
global alpha_g
global mass_2_x_g

def calculateXAccel(x_val, y_val):
    return float(- 4 * (np.pi ** 2) * (x_val / np.power((x_val ** 2) + (y_val ** 2), 3 / 2) + alpha_g * (x_val - mass_2_x_g) / np.power(((x_val - mass_2_x_g) ** 2) + (y_val ** 2), 3 / 2)))


def calculateYAccel(x_val, y_val):
    return float(- 4 * (np.pi ** 2) * y_val * (1 / np.power((x_val ** 2) + (y_val ** 2), 3 / 2) + 1 / np.power(((x_val - mass_2_x_g) ** 2) + (y_val ** 2), 3 / 2)))

In [48]:
def eulerMethod(mass_1_x, mass_1_y, mass_2_x, mass_2_y, alpha, moving_mass_x, moving_mass_y, moving_mass_x_prime, moving_mass_y_prime, its, delta):
    (mass_1_x, mass_1_y, mass_2_x, mass_2_y, moving_mass_x, moving_mass_y) = reorientAxes(mass_1_x, mass_1_y, mass_2_x,
                                                                                          mass_2_y, moving_mass_x, moving_mass_y)

    global alpha_g
    alpha_g = alpha
    global mass_2_x_g
    mass_2_x_g = mass_2_x

    # declare arrays that will be appended to in loop
    x = [moving_mass_x]
    y = [moving_mass_y]
    x_prime = [moving_mass_x_prime]
    y_prime = [moving_mass_y_prime]

    # We have the following equations of motion
    # x'_(n+1)=x'_n-deltat4pi^2 (x_(n+1)/(r_1^3)_(n+1) +alpha(x_(n+1)-d)/(r_2^3)_(n+1))
    # y'_(n+1)=y'_n-deltat4pi^2 y_(n+1) (1/(r_1^3)_(n+1) +alpha/(r_2^3)_(n+1))

    for n in range(its):
        # calculate next x, y, x', and y'
        x_n_plus_one = x[n] + delta * x_prime[n]
        x.append(x_n_plus_one)
        y_n_plus_one = y[n] + delta * y_prime[n]
        y.append(y_n_plus_one)

        x_prime_n_plus_one = x_prime[n] + delta * \
            calculateXAccel(x[n + 1], y[n + 1])
        x_prime.append(x_prime_n_plus_one)

        y_prime_n_plus_one = y_prime[n] + delta * \
            calculateYAccel(x[n + 1], y[n + 1])
        y_prime.append(y_prime_n_plus_one)

    colorline(x, y)

    masses = [[mass_1_x, mass_2_x, moving_mass_x],
              [mass_1_y, mass_2_y, moving_mass_y]]
    colors = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]])

    plt.scatter(masses[0], masses[1], c=colors / 255)

    labels = ["m1", "m2", "m"]
    for i, txt in enumerate(labels):
        plt.annotate(txt, (masses[0][i], masses[1][i]))

    plt.show()

In [52]:
def rungeKutta(mass_1_x, mass_1_y, mass_2_x, mass_2_y, alpha, moving_mass_x, moving_mass_y, moving_mass_x_prime, moving_mass_y_prime, its, delta):
    (mass_1_x, mass_1_y, mass_2_x, mass_2_y, moving_mass_x, moving_mass_y) = reorientAxes(mass_1_x, mass_1_y, mass_2_x,
                                                                                          mass_2_y, moving_mass_x, moving_mass_y)

    global alpha_g
    alpha_g = alpha
    global mass_2_x_g
    mass_2_x_g = mass_2_x

    # declare arrays that will be appended to in loop
    x = [moving_mass_x]
    y = [moving_mass_y]
    x_prime = [moving_mass_x_prime]
    y_prime = [moving_mass_y_prime]

    # we have y_(n+1)=y_n+deltat/6(k_1+2k_2+2k_3+k_4)

    for n in range(its):
        # calculate next x, y, x', and y'
        x_1 = delta * x_prime[n]
        y_1 = delta * y_prime[n]
        x_prime_1 = delta * calculateXAccel(x[n], y[n])
        y_prime_1 = delta * calculateYAccel(x[n], y[n])

        x_2 = delta * (x_prime[n] + x_prime_1 / 2)
        y_2 = delta * (y_prime[n] + y_prime_1 / 2)
        x_prime_2 = delta * calculateXAccel(x[n] + x_1 / 2, y[n] + y_1 / 2)
        y_prime_2 = delta * calculateYAccel(x[n] + x_1 / 2, y[n] + y_1 / 2)

        x_3 = delta * (x_prime[n] + x_prime_2 / 2)
        y_3 = delta * (y_prime[n] + y_prime_2 / 2)
        x_prime_3 = delta * calculateXAccel(x[n] + x_2 / 2, y[n] + y_2 / 2)
        y_prime_3 = delta * calculateYAccel(x[n] + x_2 / 2, y[n] + y_2 / 2)

        x_4 = delta * (x_prime[n] + x_prime_3)
        y_4 = delta * (y_prime[n] + y_prime_3)
        x_prime_4 = delta * calculateXAccel(x[n] + x_3, y[n] + y_3)
        y_prime_4 = delta * calculateYAccel(x[n] + x_3, y[n] + y_3)

        x_n_plus_one = x[n] + 1 / 6 * (x_1 + 2 * x_2 + 2 * x_3 + x_4)
        y_n_plus_one = y[n] + 1 / 6 * (y_1 + 2 * y_2 + 2 * y_3 + y_4)
        x_prime_n_plus_one = x_prime[n] + 1 / 6 * \
            (x_prime_1 + 2 * x_prime_2 + 2 * x_prime_3 + x_prime_4)
        y_prime_n_plus_one = y_prime[n] + 1 / 6 * \
            (y_prime_1 + 2 * y_prime_2 + 2 * y_prime_3 + y_prime_4)

        x.append(x_n_plus_one)
        y.append(y_n_plus_one)
        x_prime.append(x_prime_n_plus_one)
        y_prime.append(y_prime_n_plus_one)

    # print(x, y, x_prime, y_prime)

    colorline(x, y)

    masses = [[mass_1_x, mass_2_x, moving_mass_x],
              [mass_1_y, mass_2_y, moving_mass_y]]
    colors = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]])

    plt.scatter(masses[0], masses[1], c=colors / 255)

    labels = ["m1", "m2", "m"]
    for i, txt in enumerate(labels):
        plt.annotate(txt, (masses[0][i], masses[1][i]))

    plt.show()

In [53]:
def yoshida4thOrder(mass_1_x, mass_1_y, mass_2_x, mass_2_y, alpha, moving_mass_x,
                    moving_mass_y, moving_mass_x_prime, moving_mass_y_prime, its, delta):
    (mass_1_x, mass_1_y, mass_2_x, mass_2_y, moving_mass_x, moving_mass_y) = reorientAxes(mass_1_x, mass_1_y, mass_2_x,
                                                                                          mass_2_y, moving_mass_x, moving_mass_y)

    global alpha_g
    alpha_g = alpha
    global mass_2_x_g
    mass_2_x_g = mass_2_x

    # declare arrays that will be appended to in loop
    x = [moving_mass_x]
    y = [moving_mass_y]
    x_prime = [moving_mass_x_prime]
    y_prime = [moving_mass_y_prime]

    # Yoshida's 4th Order equation

    # coefficients that stay the same each iteration
    beta = np.power(2, 1 / 3)
    c_1 = 1 / (2 * (2 - beta))
    c_2 = (1 - beta) / (2 * (2 - beta))
    c_3 = c_2
    c_4 = c_1

    d_1 = 1 / (2 - beta)
    d_2 = -beta / (2 - beta)
    d_3 = d_1
    d_4 = 0

    for n in range(its):
        x_1 = x[n] + c_1 * x_prime[n] * delta
        y_1 = y[n] + c_1 * y_prime[n] * delta

        x_prime_1 = x_prime[n] + d_1 * calculateXAccel(x_1, y_1) * delta
        y_prime_1 = y_prime[n] + d_1 * calculateYAccel(x_1, y_1) * delta

        x_2 = x_1 + c_2 * x_prime_1 * delta
        y_2 = y_1 + c_2 * y_prime_1 * delta

        x_prime_2 = x_prime_1 + d_2 * calculateXAccel(x_2, y_2) * delta
        y_prime_2 = y_prime_1 + d_2 * calculateYAccel(x_2, y_2) * delta

        x_3 = x_2 + c_3 * x_prime_2 * delta
        y_3 = y_2 + c_3 * y_prime_2 * delta

        x_prime_3 = x_prime_2 + d_3 * calculateXAccel(x_3, y_3) * delta
        y_prime_3 = y_prime_2 + d_3 * calculateYAccel(x_3, y_3) * delta

        x_4 = x_3 + c_4 * x_prime_3 * delta
        y_4 = y_3 + c_4 * y_prime_3 * delta

        x_prime_4 = x_prime_3 + d_4 * calculateXAccel(x_4, y_4) * delta
        y_prime_4 = y_prime_3 + d_4 * calculateYAccel(x_4, y_4) * delta

        x.append(x_4)
        y.append(y_4)
        x_prime.append(x_prime_4)
        y_prime.append(y_prime_4)

    colorline(x, y)

    masses = [[mass_1_x, mass_2_x, moving_mass_x],
              [mass_1_y, mass_2_y, moving_mass_y]]
    colors = np.array([[255, 0, 0], [0, 255, 0], [0, 0, 255]])

    plt.scatter(masses[0], masses[1], c=colors / 255)

    labels = ["m1", "m2", "m"]
    for i, txt in enumerate(labels):
        plt.annotate(txt, (masses[0][i], masses[1][i]))

    plt.show()

In [54]:
style = {'description_width': 'initial'}

interact_manual(eulerMethod, mass_1_x=widgets.FloatText(value=0), mass_1_y=widgets.FloatText(value=0), mass_2_x=widgets.FloatText(value=0.2), mass_2_y=widgets.FloatText(value=0), alpha=widgets.BoundedFloatText(min=0, max=50, value=1, step=0.0001), moving_mass_x=widgets.FloatText(value=1, style=style), moving_mass_y=widgets.FloatText(value=0, style=style), moving_mass_x_prime=widgets.FloatText(value=0, style=style), moving_mass_y_prime=widgets.FloatText(value=2*np.pi, style=style), its=widgets.BoundedIntText(min=1, max=100000, value=400), delta=widgets.BoundedFloatText(min=0, max=1.0, value=0.005, readout_format=".6f", step=0.0001))
interact_manual(rungeKutta, mass_1_x=widgets.FloatText(value=0), mass_1_y=widgets.FloatText(value=0), mass_2_x=widgets.FloatText(value=0.2), mass_2_y=widgets.FloatText(value=0), alpha=widgets.BoundedFloatText(min=0, max=50, value=1, step=0.0001), moving_mass_x=widgets.FloatText(value=1, style=style), moving_mass_y=widgets.FloatText(value=0, style=style), moving_mass_x_prime=widgets.FloatText(value=0, style=style), moving_mass_y_prime=widgets.FloatText(value=2*np.pi, style=style), its=widgets.BoundedIntText(min=1, max=100000, value=400), delta=widgets.BoundedFloatText(min=0, max=1.0, value=0.005, readout_format=".6f", step=0.0001))
interact_manual(yoshida4thOrder, mass_1_x=widgets.FloatText(value=0), mass_1_y=widgets.FloatText(value=0), mass_2_x=widgets.FloatText(value=0.2), mass_2_y=widgets.FloatText(value=0), alpha=widgets.BoundedFloatText(min=0, max=50, value=1, step=0.0001), moving_mass_x=widgets.FloatText(value=1, style=style), moving_mass_y=widgets.FloatText(value=0, style=style), moving_mass_x_prime=widgets.FloatText(value=0, style=style), moving_mass_y_prime=widgets.FloatText(value=2*np.pi, style=style), its=widgets.BoundedIntText(min=1, max=100000, value=400), delta=widgets.BoundedFloatText(min=0, max=1.0, value=0.005, readout_format=".6f", step=0.0001))


interactive(children=(FloatText(value=0.0, description='mass_1_x'), FloatText(value=0.0, description='mass_1_y…

interactive(children=(FloatText(value=0.0, description='mass_1_x'), FloatText(value=0.0, description='mass_1_y…

interactive(children=(FloatText(value=0.0, description='mass_1_x'), FloatText(value=0.0, description='mass_1_y…

<function __main__.yoshida4thOrder(mass_1_x, mass_1_y, mass_2_x, mass_2_y, alpha, moving_mass_x, moving_mass_y, moving_mass_x_prime, moving_mass_y_prime, its, delta)>